<a href="https://colab.research.google.com/github/ha191180/practice-BERT/blob/main/bert_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library installation and importing

In [1]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0
import torch
from transformers import BertJapaneseTokenizer, BertModel

     |████████████████████████████████| 2.1 MB 5.2 MB/s 
     |████████████████████████████████| 486 kB 46.8 MB/s 
     |████████████████████████████████| 13.4 MB 196 kB/s 
     |████████████████████████████████| 3.3 MB 17.3 MB/s 
     |████████████████████████████████| 895 kB 51.0 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=555994be60f8d356dc59069e7dd476ba4843ae4ef207286c255900cd309cf7b3
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic


# Tokenize

**Load pretrained Japanese model**

In [2]:
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

**Example of torkenize**

In [8]:
tokenizer.tokenize('明日は自然言語処理の勉強をしよう。')

['明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '。']

This is how bert tokenize.

In [9]:
tokenizer.tokenize('明日はマシンラーニングの勉強をしよう。')

['明日', 'は', 'マシン', '##ラー', '##ニング', 'の', '勉強', 'を', 'しよ', 'う', '。']

Subword dividing.

In [10]:
tokenizer.tokenize('機械学習を中国語にすると机器学习だ。')

['機械', '学習', 'を', '中国', '語', 'に', 'する', 'と', '机', '器', '学', '[UNK]', 'だ', '。']

The behavior of unknown words.

実際のところ、文章そのもので扱うことは少ないので、そのままIDに変換し、CLSとSEPを文頭と文末に加える。

In [11]:
input_ids = tokenizer.encode('明日は自然言語処理の勉強をしよう') # get ids
print(input_ids)
tokenizer.convert_ids_to_tokens(input_ides) # decode

入力層の数は固定なので、入力するトークンについてもその数を調整する必要がある。
入力トークン列の長さのことを**系列長**という。
系列長が、揃える長さに満たない場合、特殊トークンPADを使うことができる。

In [14]:
text = '明日の天気は晴れだ。'
encoding = tokenizer(text, max_length=12, padding='max_length', truncation=True)
print('# encoding:')
print(encoding)

tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print('# tokens:')
print(tokens)

# encoding:
{'input_ids': [2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]}
# tokens:
['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]']


テキストはリストで入力することも可能である。
リストで入力する場合、リストの系列長の最大長に揃えるなど、便利な機能が利用できるようになる。


In [16]:
text_list = ['明日の天気は晴れだ。', 'パソコンが急に動かなくなった。']
encoding = tokenizer(text_list, padding='longest')
print('# encoding:')
print(encoding)

# encoding:
{'input_ids': [[2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0], [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 8, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


出力がthorch.Tensor型であると助かるので、tokenizerの引数にreturn_tensors = 'pt'を追加する。

In [18]:
text_list = ['明日の天気は晴れだ。', 'パソコンが急に動かなくなった。']
encoding = tokenizer(text_list, padding='longest', return_tensors = 'pt')
print('# encoding:')
print(encoding)

# encoding:
{'input_ids': tensor([[    2, 11475,     5, 11385,     9, 16577,    75,     8,     3,     0,
             0],
        [    2,  6311,    14,  1132,     7, 16084,   332,    58,    10,     8,
             3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


# BERTモデル
Transformersを用いてモデルのロードを行う。


In [20]:
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
bert = BertModel.from_pretrained(model_name)

bert = bert.cuda()
print(bert.config)

BertConfig {
  "_name_or_path": "cl-tohoku/bert-base-japanese-whole-word-masking",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.5.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}





```
encoding = { k: v.cuda() for k ,v in encoding.items()} # cuda-ing value of encoding 
```
 でencodingをGPUに載せる。
```
output = bert( **encoding )
```
は、
```
output = bert(
  input_ids = encoding['input_ids],
  attention_mask = encoding['attention_mask'],
  token_type_ids = encoding['token_type_ids']
```
と同義。

torch.no_grad()すると、途中計算を保存しなくなって速い。

In [27]:
text_list = [
             '明日は自然言語処理の勉強をしよう。',
             '明日はマシンラーニングの勉強をしよう。'
]

encoding = tokenizer(
    text_list,
    max_length=32,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

encoding = { k: v.cuda() for k ,v in encoding.items()} # cuda-ing value of encoding 
with torch.no_grad():
  output = bert( **encoding )
  last_hidden_state  = output.last_hidden_state
  print(last_hidden_state.size())

torch.Size([2, 32, 768])
